#  Перед вами набор данных с описаниями химических свойств стекла

Взять его можно вот тут: https://gist.github.com/hydrophis-spiralis/8899e6a3b49deeb2440c42adeb1fc18e

Столбцы кроме последнего - нюансы химического состава.
Последний столбец Type - тип стекла.

In [1]:
import pandas as pd
import numpy as np
df = pd.read_csv('glass.csv')

# 1. Сколько образцов разных видов стекла в таблице?

In [3]:
q = df['Type'].value_counts()
print(q)

buildingwindowsnonfloatprocessed    76
buildingwindowsfloatprocessed       70
headlamps                           29
vehiclewindowsfloatprocessed        17
containers                          13
tableware                            9
Name: Type, dtype: int64


# 2. Найдите средние значения химических параметров для каждого типа стекла

In [4]:
df.groupby('Type').mean()
print(df.groupby('Type').mean())

                                        RI         Na        Mg        Al  \
Type                                                                        
buildingwindowsfloatprocessed     1.518718  13.242286  3.552429  1.163857   
buildingwindowsnonfloatprocessed  1.518619  13.111711  3.002105  1.408158   
containers                        1.518928  12.827692  0.773846  2.033846   
headlamps                         1.517116  14.442069  0.538276  2.122759   
tableware                         1.517456  14.646667  1.305556  1.366667   
vehiclewindowsfloatprocessed      1.517964  13.437059  3.543529  1.201176   

                                         Si         K         Ca        Ba  \
Type                                                                         
buildingwindowsfloatprocessed     72.619143  0.447429   8.797286  0.012714   
buildingwindowsnonfloatprocessed  72.598026  0.521053   9.073684  0.050263   
containers                        72.366154  1.470000  10.123846  0.187

# 3. Найдите максимальную разницу между содержанием Si разных типов стекла. Постройте квадратную таблицу, в которой столбцы и строки - типы стекла, а в ячейках будут находиться значения абсолютной разницы.

In [5]:
x = df.groupby('Type').max().Si
y = df.groupby('Type').min().Si
print(x-y)

df1 = pd.DataFrame([['buildingwindowsfloatprocessed', 2.35],
                    ['buildingwindowsnonfloatprocessed', 4.64],
                    ['containers', 3.99],
                    ['headlamps', 4.92],
                    ['tableware', 3.04],
                    ['ehiclewindowsfloatprocessed', 1.65]],
                    columns = ['Type_new','Si_difference'])
print(df1)

Type
buildingwindowsfloatprocessed       2.35
buildingwindowsnonfloatprocessed    4.64
containers                          3.99
headlamps                           4.92
tableware                           3.04
vehiclewindowsfloatprocessed        1.65
Name: Si, dtype: float64
                           Type_new  Si_difference
0     buildingwindowsfloatprocessed           2.35
1  buildingwindowsnonfloatprocessed           4.64
2                        containers           3.99
3                         headlamps           4.92
4                         tableware           3.04
5       ehiclewindowsfloatprocessed           1.65


# 4. Как часто Барий(Ba) используется(>0) вместе с Магнием(Mg)(>0)?

In [6]:
z = df[(df['Ba'] > 0) & (df['Mg'] > 0)]
print('Барий используется вместе с магнием:', len(z))

Барий используется вместе с магнием: 15


# 5. Правда ли, что в стекле, которое используют для посуды(tableware), в среднем меньше кальция(Ca), чем в стекле для окон(Тип начинается с building window)?

In [7]:
t = df[df['Type'] == 'tableware']
w = pd.concat([df[df['Type'] == 'buildingwindowsfloatprocessed'], df[df['Type'] == 'buildingwindowsnonfloatprocessed']], ignore_index=True)
if t['Ca'].mean()>w['Ca'].mean():
    print('Кальция больше в')
    print('Tableware')
else:
    print('Windows')

Кальция больше в
Tableware


# 6. Ri - Refractive index - коэффициент преломления. Расположите другие химические параметры по убыванию корреляции с Ri. Попробуйте корреляции Пирсона и Спирмена.

In [8]:
n = df['RI'].corr(df['Na'])
m = df['RI'].corr(df['Mg'])
a = df['RI'].corr(df['Al'])
s = df['RI'].corr(df['Si'])
k = df['RI'].corr(df['K'])
c = df['RI'].corr(df['Ca'])
b = df['RI'].corr(df['Ba'])
f = df['RI'].corr(df['Fe'])
correlation = [n, m, a, s, k, c, b, f]
correlation.sort(reverse = True)
print('Корреляция RI и химических параметров поубыванию:', correlation)

Корреляция RI и химических параметров поубыванию: [0.810402696340087, 0.14300960927754583, -0.00038601891220849013, -0.12227403932819374, -0.19188537903890265, -0.28983271114396114, -0.40732603408139234, -0.5420521997085236]


# 7. Сколько образцов одновременно имеют Ri и Na больше медианной?

In [9]:
RIM = df['RI'].median()
NaM = df['Na'].median()
med = df[(df['RI'] > RIM) & (df['Na'] > NaM)]

print('количество образцов одновременно:', len(med))

количество образцов одновременно: 61


# 8. Разделите типы стекол windows по подтипам, каждый отдельный подтип вынесите в отдельную колонку в виде индикатора. Например, если в столбце Type есть building, то добавить столбец building и для таких значений поставить 1, в иных случаях - 0.

In [2]:
df.loc[:, 'vehicle'] = 0
df.loc[:, 'building'] = 0
for i in range(len(df)):
    if df.Type[i] == 'vehiclewindowsfloatprocessed':
        df.loc[i, 'vehicle'] = 1
    if df.Type[i] == 'buildingwindowsfloatprocessed' or df.Type[i] == 'buildingwindowsnonfloatprocessed':
        df.loc[i, 'building'] = 1
print(df)

          RI     Na    Mg    Al     Si     K    Ca    Ba   Fe  \
0    1.52101  13.64  4.49  1.10  71.78  0.06  8.75  0.00  0.0   
1    1.51761  13.89  3.60  1.36  72.73  0.48  7.83  0.00  0.0   
2    1.51618  13.53  3.55  1.54  72.99  0.39  7.78  0.00  0.0   
3    1.51766  13.21  3.69  1.29  72.61  0.57  8.22  0.00  0.0   
4    1.51742  13.27  3.62  1.24  73.08  0.55  8.07  0.00  0.0   
..       ...    ...   ...   ...    ...   ...   ...   ...  ...   
209  1.51623  14.14  0.00  2.88  72.61  0.08  9.18  1.06  0.0   
210  1.51685  14.92  0.00  1.99  73.06  0.00  8.40  1.59  0.0   
211  1.52065  14.36  0.00  2.02  73.42  0.00  8.44  1.64  0.0   
212  1.51651  14.38  0.00  1.94  73.61  0.00  8.48  1.57  0.0   
213  1.51711  14.23  0.00  2.08  73.36  0.00  8.62  1.67  0.0   

                              Type  vehicle  building  
0    buildingwindowsfloatprocessed        0         1  
1    buildingwindowsfloatprocessed        0         1  
2    buildingwindowsfloatprocessed        0        

# 9. Применив метод ближайшего соседа построить классификаторы типов стекла, в котором признаки - химические признаки, а предсказываемая переменная - типы стекол по стобцам из вопроса №8.